In [ ]:
# We clone all the code and tools from GitHub to this Notebook
!git clone -b 2-vdb https://github.com/kasparvonbeelen/heritageweaver.git
!sh /content/heritageweaver/create_env.sh

In [ ]:
# # we need to restart the session
# # to load the updated Pillow version
import os
os.kill(os.getpid(), 9)

In [2]:
import chromadb
import json
import time
import pandas as pd
import ipyannotations.generic
from chromadb.utils.data_loaders import ImageLoader
from heritageweaver.weavingtools.annotation_tools import *
from heritageweaver.weavingtools.weaving_tools import *
from pathlib import Path

/Users/kasparbeelen/anaconda3/envs/heritageweaver/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
collection_db = load_db("ce_comms_db",'google/siglip-base-patch16-224')
collection_df = pd.read_csv('/content/heritageweaver/data/communications_data.csv')
out_path = Path('/content/annotations')
out_path.mkdir(exist_ok=True)

(6020, 22026)

In [ ]:
annotator = 'KB'
top_n = 10

# Retrieve records


## Experiment 1
- Modality prompt: Text
- Modality search: Image and Text embedding

In [ ]:
query = 'blue telephone'
experiment_id = '1'

In [ ]:
# retrieve records
results = collection_db.query(query_texts=[query], n_results=top_n)
query_df = get_query_results(results, collection_df, source='img_url')
inputs = list(query_df.to_records())
print(f'Please go to the Annotations section of the Notebook to label the {len(inputs)} retrieved records...')

## Experiment 2
- Modality prompt: Text
- Modality search: Image embedding

In [ ]:
# define your query
query = 'blue telephone'
experiment_id = '2'

In [ ]:
# retrieve records
results = collection_db.query(query_texts=[query],where={'input_modality':'image'}, n_results=top_n)
query_df = get_query_results(results, collection_df, source='img_url')
inputs = list(query_df.to_records())
print(f'Please go to the Annotations section of the Notebook to label the {len(inputs)} retrieved records...')

# Experiment 3
- Modality prompt: Text
- Modality search: Text embedding

In [ ]:
# define your query
query = 'blue telephone'
experiment_id = '3'

In [ ]:
# retrieve records
results = collection_db.query(query_texts=[query],where={'input_modality':'text'}, n_results=top_n)
query_df = get_query_results(results, collection_df, source='img_url')
inputs = list(query_df.to_records())
print(f'Please go to the Annotations section of the Notebook to label the {len(inputs)} retrieved records...')

# Experiment 4
- Modality prompt: Image
- Modality search: Image embedding

In [2]:
query = 'https://framemark.vam.ac.uk/collections/2016HY8905/full/1400,/0/default.jpg'
experiment_id = '4'
query_img = Image.open(requests.get(query, stream=True).raw).convert("RGB")
query_img 

In [ ]:
# retrieve records
results = collection_db.query(query_images=[np.array(query_img)],where={'input_modality':'image'}, n_results=top_n)
query_df = get_query_results(results, collection_df, source='img_url')
inputs = list(query_df.to_records())
print(f'Please go to the Annotations section of the Notebook to label the {len(inputs)} retrieved records...')

## Experiment 5
- Modality prompt: Text
- Modality search: Text string

In [ ]:
# formulate query
query = 'blue telephone'

document_filter = {
    "$and": [
        {"$contains": 'blue'},
        {"$contains": 'telephone'},

    ]
}
experiment_id = '5'

In [ ]:
# retrieve records
results = collection_db.query(query_texts=[query],where_document=document_filter, n_results=top_n)
query_df = get_query_results(results, collection_df, source='img_url')
inputs = list(query_df.to_records())
print(f'Please go to the Annotations section of the Notebook to label the {len(inputs)} retrieved records...')

# Annotate

In [ ]:
labels = []

widget = ipyannotations.generic.ClassLabeller(
        options=['relevant', 'not relevant'], allow_freetext=False,
        display_function=plot_by_record)


def store_annotations(entity_annotation):
    labels.append(entity_annotation)
    try:
        widget.display(inputs.pop(0))
    except IndexError:
        print("Finished.")
        
widget.on_submit(store_annotations)
widget.display(inputs.pop(0))
widget

# Save Annotations

In [ ]:
query_df['labels'] = labels[:top_n]
query_df['query'] = query
query_df['modality'] = experiment_id
query_df['annotator'] = annotator
query_df.to_csv(out_path / f'{experiment_id}_{annotator}_{time.time()}.csv')

# Fin.